<a href="https://colab.research.google.com/github/hcarrusca/diabetic_patient_trend/blob/main/Residence_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load the CSV
df = pd.read_csv('diabetic_data.csv')

# Preview data
print("Shape:", df.shape)
df.head()

Shape: (101766, 50)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
# Count of missing or unknown values
df.replace('?', pd.NA, inplace=True)
df.replace('Unknown/Invalid', pd.NA, inplace=True)
df.replace(' ', pd.NA, inplace=True)

missing_values = df.isna().sum()
missing_values[missing_values > 0]


,0
race,2273
gender,3
weight,98569
payer_code,40256
medical_specialty,49949
diag_1,21
diag_2,358
diag_3,1423
max_glu_serum,96420
A1Cresult,84748


In [ ]:
# Drop columns with too many unknowns
df_cleaned = df.drop(columns=['weight', 'payer_code', 'medical_specialty','max_glu_serum','A1Cresult'], errors='ignore')

df_cleaned.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
# Dropping the rows with missing values on the 'race' field

df_cleaned = df_cleaned.dropna(subset=['race'])
df_cleaned.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
df_cleaned.shape

(99493, 45)

In [ ]:
# Check data types for each column
df_cleaned.dtypes

for col in df_cleaned.columns:
   print(f"{col}:{df_cleaned[col].dtype}")



encounter_id:int64
patient_nbr:int64
race:object
gender:object
age:object
admission_type_id:int64
discharge_disposition_id:int64
admission_source_id:int64
time_in_hospital:int64
num_lab_procedures:int64
num_procedures:int64
num_medications:int64
number_outpatient:int64
number_emergency:int64
number_inpatient:int64
diag_1:object
diag_2:object
diag_3:object
number_diagnoses:int64
metformin:object
repaglinide:object
nateglinide:object
chlorpropamide:object
glimepiride:object
acetohexamide:object
glipizide:object
glyburide:object
tolbutamide:object
pioglitazone:object
rosiglitazone:object
acarbose:object
miglitol:object
troglitazone:object
tolazamide:object
examide:object
citoglipton:object
insulin:object
glyburide-metformin:object
glipizide-metformin:object
glimepiride-pioglitazone:object
metformin-rosiglitazone:object
metformin-pioglitazone:object
change:object
diabetesMed:object
readmitted:object


,0
encounter_id,int64
patient_nbr,int64
race,object
gender,object
age,object
admission_type_id,int64
discharge_disposition_id,int64
admission_source_id,int64
time_in_hospital,int64
num_lab_procedures,int64


In [ ]:
cat_cols = df_cleaned.select_dtypes(include='object').columns

for col in cat_cols:
    print(f"\nUnique values in '{col}':")
    print(df_cleaned[col].unique())


Unique values in 'race':
['Caucasian' 'AfricanAmerican' 'Other' 'Asian' 'Hispanic']

Unique values in 'gender':
['Female' 'Male' <NA>]

Unique values in 'age':
['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']

Unique values in 'diag_1':
['250.83' '276' '648' '8' '197' '414' '428' '398' '434' '250.7' '157'
 '518' '999' '410' '682' '402' 'V57' '189' '786' '427' '996' '277' '584'
 '462' '473' '411' '174' '486' '998' '511' '432' '626' '295' '196' '250.6'
 '182' '845' '423' '808' '250.4' '722' '403' '250.11' '784' '707' '440'
 '151' '715' '997' '198' '564' '38' '590' '578' '250.32' '433' '569' '185'
 '536' '255' '250.13' '599' '558' '574' '491' '560' '244' '250.03' '577'
 '730' '188' '824' '250.8' '332' '562' '291' '296' '510' '401' '263' '438'
 '70' '250.02' '493' '642' '571' '738' '593' '618' '250.42' '807' '456'
 '446' '572' '575' '250.41' '820' '515' '780' '250.22' '995' '235'
 '250.82' '721' '787' '556' '162' '724' '282' '514' 'V55

In [ ]:
## Encoding Variables

# Binary or ordinal
df_cleaned['gender'] = df_cleaned['gender'].astype(str)
df_cleaned['gender'] = LabelEncoder().fit_transform(df_cleaned['gender'])

df_cleaned['readmitted'] = df_cleaned['readmitted'].map({'NO': 0, '>30': 1, '<30': 2})
df_cleaned['diabetesMed'] = df_cleaned['diabetesMed'].map({'No': 0, 'Yes': 1})
df_cleaned['change'] = df_cleaned['change'].map({'No': 0, 'Ch': 1})

# Ordinal: age group
age_order = {'[0-10)': 0, '[10-20)': 1, '[20-30)': 2, '[30-40)': 3, '[40-50)': 4,
             '[50-60)': 5, '[60-70)': 6, '[70-80)': 7, '[80-90)': 8, '[90-100)': 9}
df_cleaned['age'] = df_cleaned['age'].map(age_order)

In [ ]:
## One-hot encoding Race

df_cleaned = pd.get_dummies(df_cleaned, columns=['race'], drop_first=True)

In [ ]:
## Encoding medical columns as ordinal variables

med_columns = [
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
    'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
    'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
    'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
    'insulin', 'glyburide-metformin', 'glipizide-metformin',
    'glimepiride-pioglitazone', 'metformin-rosiglitazone',
    'metformin-pioglitazone'
]

med_mapping = {'No': 0, 'Down': 1, 'Steady': 2, 'Up': 3}

for col in med_columns:
    df_cleaned[col] = df_cleaned[col].map(med_mapping)


In [ ]:
## Categorizing Diagnosis before encoding

for col in ['diag_1', 'diag_2', 'diag_3']:
    df_cleaned[col] = df_cleaned[col].astype(str).str[:3]

def categorize_diag(code):
    try:
        code = float(code)
        if code == 250:
            return 'Diabetes'
        elif 390 <= code <= 459 or code == 785:
            return 'Circulatory'
        elif 460 <= code <= 519 or code == 786:
            return 'Respiratory'
        elif 520 <= code <= 579 or code == 787:
            return 'Digestive'
        elif 800 <= code <= 999:
            return 'Injury'
        elif 290 <= code <= 319:
            return 'Mental Health'
        else:
            return 'Other'
    except:
        return 'Unknown'

for col in ['diag_1', 'diag_2', 'diag_3']:
    df_cleaned[col] = df_cleaned[col].map(categorize_diag)

In [ ]:
## One-hot Encoding Diagnosis Variables

df_cleaned = pd.get_dummies(df_cleaned, columns=['diag_1', 'diag_2', 'diag_3'], drop_first=True)

In [ ]:
df_cleaned.head()

,encounter_id,patient_nbr,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,diag_2_Other,diag_2_Respiratory,diag_2_Unknown,diag_3_Diabetes,diag_3_Digestive,diag_3_Injury,diag_3_Mental Health,diag_3_Other,diag_3_Respiratory,diag_3_Unknown
0,2278392,8222157,1,0,6,25,1,1,41,0,...,False,False,True,False,False,False,False,False,False,True
1,149190,55629189,1,1,1,1,7,3,59,0,...,False,False,False,False,False,False,False,True,False,False
2,64410,86047875,1,2,1,1,7,2,11,5,...,False,False,False,False,False,False,False,False,False,True
3,500364,82442376,2,3,1,1,7,2,44,1,...,False,False,False,False,False,False,False,False,False,False
4,16680,42519267,2,4,1,1,7,1,51,0,...,True,False,False,True,False,False,False,False,False,False
